<a href="https://colab.research.google.com/github/TerriblePepito/TerriblePepitostest/blob/main/changefontcolor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip -q install svgpathtools lxml

import re
from pathlib import Path
from lxml import etree
from svgpathtools import parse_path

# ====== Réglages ======
INPUT_DIR  = Path("/content/drive/MyDrive/testinput")
OUTPUT_DIR = Path("/content/drive/MyDrive/testoutput")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

YELLOW_HEXES = {"#ffff00", "#ff0", "yellow"}  # variantes courantes
TARGET_TEXT_COLOR = "#000000"                 # noir

# ====== Utilitaires couleur/style ======
def parse_style(style_str: str):
    d = {}
    if not style_str:
        return d
    for part in style_str.split(";"):
        part = part.strip()
        if not part or ":" not in part:
            continue
        k, v = part.split(":", 1)
        d[k.strip()] = v.strip()
    return d

def style_to_str(d: dict):
    return ";".join(f"{k}:{v}" for k, v in d.items() if v is not None)

def norm_color(c: str):
    if not c:
        return None
    c = c.strip().lower()
    # normalise quelques formats
    if c.startswith("rgb(") and c.endswith(")"):
        # rgb(255,255,0) -> #ffff00
        nums = re.findall(r"\d+", c)
        if len(nums) == 3:
            r, g, b = map(int, nums)
            return "#{:02x}{:02x}{:02x}".format(r, g, b)
    return c

def get_fill(elem):
    # fill peut être dans attribut ou dans style
    fill = elem.get("fill")
    if fill:
        return norm_color(fill)
    style = parse_style(elem.get("style"))
    if "fill" in style:
        return norm_color(style["fill"])
    return None

def set_text_fill(elem, new_color="#000000"):
    # Essaie d'écrire dans style si présent, sinon fill direct.
    style = parse_style(elem.get("style"))
    if elem.get("style") is not None or "fill" in style:
        style["fill"] = new_color
        elem.set("style", style_to_str(style))
        # si fill existe aussi, on peut l'aligner
        if elem.get("fill") is not None:
            elem.set("fill", new_color)
    else:
        elem.set("fill", new_color)

# ====== Utilitaires bbox & inclusion ======
def bbox_union(b1, b2):
    if b1 is None: return b2
    if b2 is None: return b1
    (x1, y1, x2, y2) = b1
    (a1, b1y, a2, b2y) = b2
    return (min(x1, a1), min(y1, b1y), max(x2, a2), max(y2, b2y))

def point_in_bbox(x, y, bbox):
    x1, y1, x2, y2 = bbox
    return (x1 <= x <= x2) and (y1 <= y <= y2)

def bbox_from_rect(elem):
    try:
        x = float(elem.get("x", "0"))
        y = float(elem.get("y", "0"))
        w = float(elem.get("width", "0"))
        h = float(elem.get("height", "0"))
        return (x, y, x+w, y+h)
    except:
        return None

def bbox_from_circle(elem):
    try:
        cx = float(elem.get("cx", "0"))
        cy = float(elem.get("cy", "0"))
        r  = float(elem.get("r", "0"))
        return (cx-r, cy-r, cx+r, cy+r)
    except:
        return None

def bbox_from_ellipse(elem):
    try:
        cx = float(elem.get("cx", "0"))
        cy = float(elem.get("cy", "0"))
        rx = float(elem.get("rx", "0"))
        ry = float(elem.get("ry", "0"))
        return (cx-rx, cy-ry, cx+rx, cy+ry)
    except:
        return None

def bbox_from_polygon_points(points_str):
    try:
        pts = []
        for token in re.split(r"\s+", points_str.strip()):
            if not token:
                continue
            if "," in token:
                xs, ys = token.split(",", 1)
            else:
                # parfois "x y x y"
                continue
            pts.append((float(xs), float(ys)))
        if not pts:
            # fallback format: "x1 y1 x2 y2 ..."
            nums = re.findall(r"[-+]?\d*\.?\d+", points_str)
            coords = list(map(float, nums))
            pts = list(zip(coords[0::2], coords[1::2]))
        xs = [p[0] for p in pts]
        ys = [p[1] for p in pts]
        return (min(xs), min(ys), max(xs), max(ys))
    except:
        return None

def bbox_from_path_d(d):
    try:
        p = parse_path(d)
        xmin, xmax, ymin, ymax = p.bbox()  # note: retourne xmin, xmax, ymin, ymax
        return (xmin, ymin, xmax, ymax)
    except:
        return None

def get_text_xy(text_elem):
    """
    Récupère la première coordonnée x/y du <text> ou <tspan>.
    (Cas simples : x="123" y="456" ou x="123 124 ..." )
    """
    x = text_elem.get("x")
    y = text_elem.get("y")
    if x is None or y is None:
        return None

    try:
        # si liste: "12 13 14"
        x0 = float(str(x).strip().split()[0].split(",")[0])
        y0 = float(str(y).strip().split()[0].split(",")[0])
        return (x0, y0)
    except:
        return None

# ====== Traitement d'un SVG ======
def process_one_svg(in_path: Path, out_path: Path):
    parser = etree.XMLParser(remove_comments=False, recover=True)
    tree = etree.parse(str(in_path), parser)
    root = tree.getroot()

    # Namespace SVG (souvent présent)
    # lxml gère tag avec namespace, ex: {http://www.w3.org/2000/svg}path
    def localname(tag):
        return tag.split("}")[-1] if "}" in tag else tag

    # 1) Collecter bboxes des formes jaunes
    yellow_bboxes = []
    for elem in root.iter():
        tag = localname(elem.tag)
        if tag not in {"path", "rect", "polygon", "polyline", "circle", "ellipse"}:
            continue

        fill = get_fill(elem)
        if fill is None:
            continue

        if fill in YELLOW_HEXES:
            bbox = None
            if tag == "path":
                d = elem.get("d")
                if d:
                    bbox = bbox_from_path_d(d)
            elif tag == "rect":
                bbox = bbox_from_rect(elem)
            elif tag in {"polygon", "polyline"}:
                pts = elem.get("points")
                if pts:
                    bbox = bbox_from_polygon_points(pts)
            elif tag == "circle":
                bbox = bbox_from_circle(elem)
            elif tag == "ellipse":
                bbox = bbox_from_ellipse(elem)

            if bbox is not None:
                yellow_bboxes.append(bbox)

    # 2) Parcourir les textes, si position dans une bbox jaune => texte noir
    changed = 0
    for elem in root.iter():
        tag = localname(elem.tag)
        if tag not in {"text", "tspan"}:
            continue

        xy = get_text_xy(elem)
        if not xy:
            continue
        x, y = xy

        if any(point_in_bbox(x, y, bb) for bb in yellow_bboxes):
            set_text_fill(elem, TARGET_TEXT_COLOR)
            changed += 1

    tree.write(str(out_path), encoding="utf-8", xml_declaration=True, pretty_print=True)
    return changed, len(yellow_bboxes)

# ====== Batch sur dossier ======
total_files = 0
total_texts_changed = 0

svg_files = sorted(INPUT_DIR.glob("*.svg"))
if not svg_files:
    print(f"Aucun .svg trouvé dans {INPUT_DIR}")

for f in svg_files:
    out = OUTPUT_DIR / f.name
    changed, nb_yellow = process_one_svg(f, out)
    print(f"{f.name} -> {out.name} | flèches jaunes détectées: {nb_yellow} | textes modifiés: {changed}")
    total_files += 1
    total_texts_changed += changed

print("\nTerminé.")
print(f"Fichiers traités: {total_files}")
print(f"Textes passés en noir: {total_texts_changed}")